To begin with we need to import the models we are testing, and some additional libraries/functions

In [ ]:
from .models import ClassifierWrapper, ClusteringWrapper, train_model,set_defaults,set_log_level, use_accel
from .datagen import generate_classification_data, generate_clustering_data
from sklearn.model_selection import train_test_split
import logging as LOG
set_log_level(LOG.DEBUG)
use_accel(True)

Now we define our syntethic dataset

In [2]:
classifierXY = generate_classification_data(n_samples=100_000, n_features=20, n_informative=15, n_classes=2,sparsity=0.0, random_state=42)
classifierSplit = train_test_split(classifierXY[0], classifierXY[1], test_size=0.2, random_state=42)
clusteringXY = generate_clustering_data(n_samples=100_000, n_features=50, centers=10, cluster_std=1.0, random_state=42)


Subsequently we define the model parameters and models used for testing.


In [3]:
SEED = 42

setting={
    "classifier": {
        "estimator_name": "random_forest",
        "use_scaler": True,
        "random_state": SEED,
        "n_estimators": 100,
        "max_iter": 1000,
        "probability": True,
    },
    "clustering": {
        "algorithm_name": "kmeans",
        "n_clusters": 3,
        "use_scaler": True,
        "random_state": SEED,
        "algorithm_params": None,
    },
}
set_defaults(setting)

rfc = ClassifierWrapper()
kmc = ClusteringWrapper()

In [4]:
resultClassifier=train_model(model=rfc,X=classifierSplit[0],y=classifierSplit[2],X_val=classifierSplit[1],y_val=classifierSplit[3],timing=True)
resultClustering=train_model(kmc,X=clusteringXY[0],timing=True)
print("Classifier results:", resultClassifier)
print("Clustering results:", resultClustering)



Classifier results: {'times': [47.52599168400047], 'median': 47.52599168400047, 'mean': 47.52599168400047, 'std': 0.0}
Clustering results: {'times': [0.244890927000597], 'median': 0.244890927000597, 'mean': 0.244890927000597, 'std': 0.0}
